In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import autocorrelation_plot, lag_plot
import pandas
from scipy import stats
import os
from statsmodels.graphics.tsaplots import plot_acf

/home/vgoncalves/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
filename = os.path.realpath(os.path.join(os.getcwd(), os.path.dirname('treinamento.txt'))) + '/treinamento-1.txt'

In [4]:
series = pandas.read_csv(filename,  header=None)

In [6]:
def series_to_supervised(df, n_lags, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a pandas dataframe.
        n_lags: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = df.shape[1]
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_lags, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pandas.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [36]:
D = series_to_supervised(series, 3).values

X = D[:,0:-1]
Y = D[:,-1].reshape(X.shape[0],1)

train_size = round(X.shape[0] * 0.7)
test_size = X.shape[0] - train_size
Xtr = X[0:train_size,:]
Xv = X[train_size:train_size+test_size,:]
Ytr = Y[0:train_size,:]
Yv = Y[train_size:train_size+test_size,:]